# Domherren der Personendatenbank
## Datenimport / -update WIAGweb 1

Quellen: Personendatenbank (über https://mysql.gwdg.de/)  
C:\Users\georg\Documents\projekte-doc\gso\data

- [Personendaten](#Personendaten)
- [IDs zuordnen](#IDs_zuordnen)
- [Literaturangaben](#Literaturangaben)
- [Ämter](#Ämter)

2021-11-09

In [2]:
wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

"C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"

In [3]:
cd(wds_path)

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

In [6]:
using Revise

In [7]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1317


In [8]:
using MySQL

In [9]:
using DataFrames

In [10]:
Wds = WiagDataSetup

WiagDataSetup

In [11]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg
stdin> topf&deckel


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [11]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

Datenbank anlegen: `wiag_in`  
Tabellen anlegen: siehe `domherren_create.sql`
- `cn_canon_gs`
- `cn_office_gs`
- `cn_canon_reference_gs`
- `cn_reference_gs`

## Personendaten

Betrachte nur Personen, deren Eintrag nicht gelöscht ist und die online sind. Aus einem nicht erschließbaren Grund ist `item_id` nur eindeutig, wenn aus der Tabelle `gsn` nur der Eintrag mit der kleinsten ID berücksichtigt wird.
``` sql
select gsn.nummer, item_id, gsn.deleted, items.deleted, items.status from gsn join items on gsn.item_id = items.id where items.status = 'online' and not items.deleted limit 12 offset 4000;
select count(*) from gsn join items on gsn.item_id = items.id where items.status = 'online' and not items.deleted;
```
96996
``` sql
select count(distinct(gsn.nummer)) from gsn join items on gsn.item_id = items.id where items.status = 'online' and not items.deleted;
```
96995
``` sql
create temporary table item_online as
select min(gsn.id) as gsn_tbl_id, gsn.nummer, item_id, items.deleted, items.status
from gso_in_202111.gsn join gso_in_202111.items on gsn.item_id = items.id where items.status = 'online'
group by items.id;
```
87247

In [19]:
sql = "SELECT min(gsn.id) as gsn_tbl_id, gsn.nummer as gsn_id, max(item_id) as item_id, items.status " * 
"FROM gso_in_202111.gsn JOIN gso_in_202111.items ON gsn.item_id = items.id " *
"WHERE items.status = 'online' AND NOT items.deleted " *
"GROUP BY item_id";
df_p_online = db_exec(sql);

In [20]:
size(df_p_online)

(79951, 4)

Ermittle die IDs der Domherren anhand der relevanten Stifte.

In [21]:
sql = "SELECT gs_id FROM wiag.domstift"
df_ds = db_exec(sql);

In [22]:
size(df_ds)

(34, 1)

In [23]:
str_ds = join(df_ds.gs_id, ", ")

"3498, 3492, 701, 343, 60130, 3493, 3496, 783, 675, 739, 832, 3503, 3499, 736, 676, 3488, 3501, 792, 794, 616, 628, 3491, 803, 953, 3495, 226, 3494, 679, 2066, 3489, 3500, 3487, 3490, 3502"

In [24]:
sql = "SELECT DISTINCT person_id " *
"FROM gso_in_202111.offices " *
"WHERE klosterid in (" * str_ds * ") "
df_ids_cn = db_exec(sql);

In [93]:
size(df_ids_cn)

(11342, 1)

In [25]:
str_ids_cn = join(df_ids_cn.person_id, ", ");

Lies Personendaten

In [26]:
sql = "SELECT p.id as id_person_gso, item_id, " *
"namenspraefix as prefix_name, vorname as givenname, familienname as familyname, " *
"sterbedatum as date_death, geburtsdatum as date_birth, `orden` as religious_order, " *
"anmerkungen as annotation_ed, titel as academic_title, " * 
"gndnummer as gnd_id, viaf as viaf_id, " *
"familiennamenvarianten as familyname_variant, vornamenvarianten as givenname_variant, " *
"namenszusatz as comment_name, " *
"zeitraum_von as date_hist_first, zeitraum_bis as date_hist_last " *
"FROM gso_in_202111.persons as p " *
"WHERE p.id IN (" * str_ids_cn * ") " *
"ORDER BY p.id"
df_p_cn = db_exec(sql);

In [27]:
Wds.clean_up!(df_p_cn)

In [28]:
size(df_p_cn)

(11342, 17)

Ergänze die GSN und beschränke auf Domherren, die online sind.

In [29]:
df_p_cn = innerjoin(df_p_cn, df_p_online, on = :item_id);

In [30]:
size(df_p_cn)

(7510, 20)

In [83]:
names(df_p_cn)

20-element Vector{String}:
 "id_person_gso"
 "item_id"
 "prefix_name"
 "givenname"
 "familyname"
 "date_death"
 "date_birth"
 "religious_order"
 "annotation_ed"
 "academic_title"
 "gnd_id"
 "viaf_id"
 "familyname_variant"
 "givenname_variant"
 "comment_name"
 "date_hist_first"
 "date_hist_last"
 "gsn_tbl_id"
 "gsn_id"
 "status"

In [102]:
first(df_p_cn, 6)

,id_person_gso,item_id,prefix_name,givenname,familyname,date_death,date_birth
,Int32,Int32,String,String,String,String,String
1,4586,4586,,Nikolaus,Amelung,,
2,4590,4590,von,Johann,Angern,1423,
3,4620,4620,,Lorenz,Bading,1557,
4,4623,4623,(von),Werner,Balck,,
5,4678,4678,(von),Johannes,Bellin,,
6,4684,4684,von,Werner,Berg,,


### IDs zuordnen
Lies bestehende IDs

In [31]:
sql = "SELECT id, id_person_gso FROM wiag.cn_canon_gs"
df_id_cn_wiag = db_exec(sql);

In [33]:
df_p_cn = leftjoin(df_p_cn, df_id_cn_wiag, on = :id_person_gso);

In [34]:
size(df_p_cn)

(7510, 21)

In [35]:
size(filter(ismissing, df_p_cn.id)) # zweiter Durchlauf (für alle werden IDs gefunden)

(0,)

In [109]:
7313 + 217

7530

Es gibt etliche neue Domherren, aber 20 sind auch nicht mehr da?!

In [21]:
df_cn_gone = antijoin(df_id_cn_wiag, df_p_cn, on = :id_person_gso);

In [22]:
first(df_cn_gone, 5)

,id,id_person_gso
,Int32,Int32?
1,80403,10814
2,80732,17882
3,80965,23099
4,81144,44556
5,81153,45016


Das sind zum Beispiel bereinigte Dubletten (005-03705-001).  
**R** Domherren, die aus dem aktuellen Online-Bestand der Personendatenbank herausgenommen werden, werden über WIAG nicht mehr zu finden sein, weil der Redaktionsprozess der Personendatenbank in WIAG nicht nachgebildet wird.

In [116]:
df_p_cn[100:130, [:id, :id_person_gso, :gsn_tbl_id]];

Die Daten sind nach `gsn_tbl_id` sortiert (auch die anderen beiden IDs folgen dieser Sortierung)

In [27]:
df_p_cn_mg_id = filter(:id => ismissing, df_p_cn);

In [119]:
df_p_cn_id = filter(:id => !ismissing, df_p_cn);

In [124]:
maximum(df_p_cn_id.id_person_gso), size(df_p_cn_id)

(325811, (7293, 21))

In [125]:
minimum(df_p_cn_mg_id.id_person_gso), size(df_p_cn_mg_id)

(4914, (217, 21))

Sobald hier neue IDs vergeben werden, wird die Sortierung von ID (in WIAG) und `persons.id` nicht mehr kongruent sein.

In [24]:
id_max = maximum(df_id_cn_wiag.id)

88657

In [133]:
id_max, s_cn_id = maximum(df_p_cn_id.id), size(df_p_cn_id)

(88657, (7293, 21))

In [28]:
id_person_gso = sort(df_p_cn_mg_id.id_person_gso);

In [136]:
last(df_id_cn_wiag, 8)

,id,id_person_gso
,Int32,Int32?
1,88649,325789
2,88650,325790
3,88651,325792
4,88653,325795
5,88654,325798
6,88655,325803
7,88656,325808
8,88657,325811


In [29]:
id_c = id_max + 1;
for id_gso in id_person_gso
    push!(df_id_cn_wiag, (id_c, id_gso))
    id_c += 1;
end

`df_id_cn_wiag` hat jetzt 20 Zeilen mehr als `df_p_cn`: eben jene, die in der Quelle als Dubletten erkannt wurden oder aus anderen Gründen entfernt wurden.

In [144]:
length(unique(df_id_cn_wiag.id)), length(unique(df_id_cn_wiag.id_person_gso))

(7530, 7530)

Entferne die IDs wieder, die schon zugeordnet waren und füge sie wieder ein mit dem vervollständigten Satz von IDs.

In [30]:
df_p_cn = select(df_p_cn, Not(:id));

In [181]:
size(df_p_cn)

(7510, 20)

In [31]:
df_p_cn = leftjoin(df_p_cn, df_id_cn_wiag, on = :id_person_gso);

In [147]:
size(filter(:id => ismissing, df_p_cn))

(0, 21)

In [36]:
Wds.clean_up!(df_p_cn)

In [41]:
df_p_cn[20:30, [:id, :id_person_gso, :givenname, :date_death]]

,id,id_person_gso,givenname,date_death
,Int32,Int32,SubStri…,SubStri…?
1,80020,4984,Engelbert,missing
2,80021,5049,Matthäus,missing
3,80022,5090,Joachim,1544
4,80023,5092,Jakob,missing
5,80024,5096,Peter,missing
6,80025,5135,Gebhard,missing
7,80026,5156,Heinrich,missing
8,80027,5166,Gerhard,missing
9,80028,5170,Nikolaus,missing


In [37]:
numdate_birth(date) = Wds.parsemaybe(date, :lower)
df_p_cn[!, :numdate_birth] .= numdate_birth.(df_p_cn.date_birth);

In [42]:
numdate_death(date) = Wds.parsemaybe(date, :upper)
df_p_cn[!, :numdate_death] .= numdate_death.(df_p_cn.date_death);

┌ Warning: Could not parse 
│   s = 28. August
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1068
┌ Warning: Could not parse 
│   s = Kammerkleriker
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1093


In [156]:
table_name = "cn_canon_gs"
Wds.filltable!(table_name, select(df_p_cn, Not([:gsn_tbl_id, :item_id])))

┌ Info: Rows inserted: 7510
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


7510

In [157]:
size(df_p_cn)

(7510, 23)

## Literaturangaben
Die in der Tabelle `locations` angegebenen Werte für `book_id` sind kongruent zu den IDs in `wiag.cn_reference_gs`.
``` sql
select count(*) from locations 
where book_id not in (select id from wiag.cn_reference_gs) 
and item_status = 'online' and not deleted;
```
0

Referenzwerke (`gso_in_202111.books`)
``` sql
source 'C:\Users\georg\Documents\projekte-doc\gso\data\books.sql';
```

In [15]:
table_src = "gso_in_202111.books"
sql = "SELECT id, titel AS title, autoren AS author, nummer AS number_vol, " *
"uri AS online_resource, kurztitel AS shorttitle " *
"FROM " * table_src 
df_vol = Wds.sql_df(sql);

In [13]:
size(df_vol)

(86, 5)

In [16]:
table_name = "cn_reference_gs"
Wds.filltable!(table_name, df_vol)

┌ Info: Rows inserted: 86
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1191


86

Lies die Domherren ein

In [18]:
sql = "SELECT c.id AS id_canon, p.item_id FROM cn_canon_gs AS c " *
"JOIN gso_in_202111.persons as p on p.id = c.id_person_gso"
df_cn = Wds.sql_df(sql);

In [19]:
size(df_cn)

(7510, 2)

Lies die Literaturverweise ein

In [20]:
sql = "SELECT * FROM gso_in_202111.locations " * 
"WHERE item_status = 'online' AND NOT deleted " *
"GROUP BY item_id, book_id, seiten"
df_p_ref = Wds.sql_df(sql);

In [21]:
size(df_p_ref)

(96788, 11)

Welche Domherren haben eine Kurz-Biographie?

In [22]:
is_bio(seiten) = contains(seiten, "<b>");
df_p_ref[!, :is_bio] .= is_bio.(df_p_ref.seiten);

In [23]:
sum(df_p_ref.is_bio)

33910

Übernimm für Domherren mit Kurzbiographie nur Literaturhinweise mit Kurzbiographien (`transfer_gso.sql`: 2021-04-26) Es ist freilich möglich, dass es mehrere Literaturverweise gibt, die eine Kurzbiographie enthalten.

In [34]:
df_p_ref_bio = filter(:is_bio => identity, df_p_ref);

In [29]:
size(df_p_ref_bio)

(33910, 12)

In [35]:
df_cn_ref_bio = innerjoin(df_cn, df_p_ref_bio, on = :item_id);

In [36]:
size(df_cn_ref_bio), length(unique(df_cn_ref_bio.id_canon))

((6235, 13), 5547)

In [33]:
df_cn_mg_bio = antijoin(df_cn, select(df_cn_ref_bio, :id_canon), on = :id_canon);

In [35]:
size(df_cn_mg_bio)

(1963, 2)

In [38]:
df_cn_ref_mg_bio = innerjoin(df_cn_mg_bio, df_p_ref, on = :item_id);

In [39]:
size(df_cn_ref_mg_bio)

(2418, 13)

Wandle `is_bio` in Int um.

In [38]:
transform!(df_cn_ref_bio, :is_bio => ByRow(Int) => :is_bio);

In [42]:
transform!(df_cn_ref_mg_bio, :is_bio => ByRow(Int) => :is_bio);

In [39]:
columns = [
    :id_canon => :id_canon,
    :book_id => :id_reference,
    :seiten => :page_reference,
    :is_bio => :isbio,
    :item_status => :status,
];

Schreibe die Einträge mit Kurzbiographie

In [40]:
table_name = "cn_canon_reference_gs"
Wds.filltable!(table_name, select(df_cn_ref_bio, columns), clear_table = true)

┌ Info: Rows inserted: 6235
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1191


6235

*2021-19-11* werden nicht mehr ausgegeben; siehe Vorgängerversion der Daten.  
Schreibe die Einträge ohne Kurzbiographie

In [47]:
table_name = "cn_canon_reference_gs"
Wds.filltable!(table_name, select(df_cn_ref_mg_bio, columns), clear_table = false)

┌ Info: Rows inserted: 2418
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1184


2418

## Ämter

In [12]:
sql = "SELECT person_id as id_person_gso, dioezese as diocese, bezeichnung as office_name, " * 
"von as date_start, bis as date_end, institution, ort as location, " *
"anmerkung as comment, klosterid as id_monastery " *
"FROM gso_in_202111.offices " *
"WHERE NOT deleted"
df_oc = db_exec(sql);

In [13]:
Wds.clean_up!(df_oc)

In [14]:
size(df_oc)

(69927, 9)

In [43]:
df_o_cn = innerjoin(df_oc, select(df_p_cn, :id => :id_canon, :id_person_gso), on = :id_person_gso);

In [44]:
size(df_o_cn)

(13002, 10)

`location_show` und `location` stimmen für Domherren aus der Personendatenbank überein.

In [81]:
df_o_cn[!, :location_show] .= df_o_cn.location;

In [82]:
date_start(date) = Wds.parsemaybe(date, :lower)
df_o_cn[!, :numdate_start] .= date_start.(df_o_cn.date_start);

┌ Warning: Could only find year in 
│   s = vor 1522
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1082
┌ Warning: Could only find year in 
│   s = um1220
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1082
┌ Warning: Could only find year in 
│   s = vor 1400
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1082
┌ Warning: Could only find year in 
│   s = vor 1400
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1082


In [83]:
date_start(date) = Wds.parsemaybe(date, :lower)
df_dp = df_o_cn[260:265, :]
df_dp[!, :numdate_start] .= date_start.(df_dp.date_start);

┌ Warning: Could only find year in 
│   s = vor 1522
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1082


In [54]:
sd = df_dp[3, :date_start]

"vor 1522"

In [55]:
sd[4]

' ': Unicode U+00A0 (category Zs: Separator, space)

Die Daten enthalten ein geschütztes Leerzeichen?! In solchen Fällen verwenden wir eine drei- oder vierstellige Jahreszahl, wenn wir sie finden.

In [56]:
rgpyear = "([1-9][0-9][0-9]+)"

"([1-9][0-9][0-9]+)"

In [58]:
Regex(rgpyear)

r"([1-9][0-9][0-9]+)"

In [2]:
rgxyear = r"([1-9][0-9][0-9][0-9]?)"

r"([1-9][0-9][0-9][0-9]?)"

In [3]:
rgm = match(rgxyear, "131")

RegexMatch("131", 1="131")

In [62]:
rgm[1]

"1312"

In [45]:
date_start(date) = Wds.parsemaybe(date, :lower)
df_o_cn[!, :numdate_start] .= date_start.(df_o_cn.date_start);

┌ Warning: Could only find year in 
│   s = vor 1522
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1088
┌ Warning: Could only find year in 
│   s = um1220
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1088
┌ Warning: Could only find year in 
│   s = vor 1400
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1088
┌ Warning: Could only find year in 
│   s = vor 1400
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1088


In [46]:
date_end(date) = Wds.parsemaybe(date, :upper)
df_o_cn[!, :numdate_end] .= date_end.(df_o_cn.date_end);

In [50]:
df_o_cn[20:30, [:id_canon, :office_name, :date_start, :date_end, :numdate_start, :numdate_end]]

,id_canon,office_name,date_start,date_end,numdate_start,numdate_end
,Int32,SubStri…?,SubStri…?,SubStri…?,Int64?,Int64?
1,84434,Domherr,1807,missing,1807,missing
2,84435,Domherr,1792,missing,1792,missing
3,81140,Propst,1559,1569/1571,1559,1569
4,81055,Dompropst,um 1150,missing,1145,missing
5,83575,Domherr,1873,1887,1873,1887
6,81378,Domherr,1356,missing,1356,missing
7,81407,Domherr,1732,missing,1732,missing
8,81423,Domherr,1396,missing,1396,missing
9,81439,Domherr,Ende 18. Jahrhundert,missing,1767,missing


Welche Bistümer sind in der Datenbank?

In [51]:
sql = "SELECT diocese FROM wiag.diocese"
df_dioc = db_exec(sql);

In [52]:
in_db(diocese) = ismissing(diocese) ? 0 : (in(diocese, df_dioc.diocese) ? 1 : 0)
df_o_cn[!, :diocese_in_db] .= 0;
for row in eachrow(df_o_cn)
    row[:diocese_in_db] = in_db(row[:diocese])
end

Das Domstift Krakau ist nicht in der Klosterdatenbank. Dafür sind aber die Felder `institution` und `location` befüllt.

In [53]:
table_name = "cn_office_gs"
Wds.filltable!(table_name, select(df_o_cn, Not(:id_person_gso)), clear_table = true)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1161
┌ Info: Rows inserted: 13002
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1184


13002

Was ist mit der Diözese 'Hamburg-Bremen'?